<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fibd/notebooks/ibd_rs_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install and Setup

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-6t_zd86w
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-6t_zd86w
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 21fa4d2737a3bfc9a7871c9fd14b9210e662155d
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.2 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.5.0-py3-none-any.whl size=72922 sha256=ed49b43c3023078abde146dd69388fa6b46c54da71e5d74465505968a98ae666
  Stored in directory: /tmp/pip-ephem-wheel-cache-z0rt6ykc/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

#### Setup and Configuration

In [2]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Cannot download.")
            return None

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers)
        if file_info:
            download_url = file_info['download_url']
            response = requests.get(download_url)
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
                return None
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            sha = file_info['sha']
            payload = {
                'message': 'Deleting file',
                'sha': sha,
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='ibd',
)

In [5]:
# @title Update and Filter DataFrame

def update_tickers_with_names(df, ticker_column, name_separator=','):
    """
    Update ticker codes in a DataFrame with their corresponding stock names.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    - name_separator: Separator used to join names (default is comma for multiple tickers).

    This function updates the specified column with the stock names instead of ticker codes.
    """
    # Iterate over the specified column in the DataFrame
    for index, row in df.iterrows():
        tickers = row[ticker_column].split(name_separator)  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df.at[index, ticker_column] = name_separator.join(stock_names)  # Join the names back into a string


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [27]:
# @title Rank Function

import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers

def remove_failed_tickers(tickers):
    delisted = ['BRK.B', 'LEN.B', 'BF.B', 'UHAL.B', 'BF.A', 'CWEN.A', 'HEI.A']
    invalid = ['GEV', 'SOLV', 'VLTO', 'SW', 'ARM', 'CART', 'AS', 'BIRK', 'VSTS','LOAR', 'ALAB','GRAL', 'SEG']
    return list(set(tickers) - set(delisted) - set(invalid))

def rank(code, period='2y', tickers_getter=get_tickers,
         ticker_ref='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    #tickers = [t.lstrip('$') for t in tickers]
    tickers = remove_failed_tickers(tickers)

    rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                           ticker_ref=ticker_ref)
    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Update the stock names back to the DataFrame
    #update_tickers_with_names(rank_stock, 'Ticker')
    #update_tickers_with_names(rank_indust, 'Tickers')

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')
    remove_ticker_suffix(rank_indust, 'Tickers')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    for df, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        github.upload_df_as_csv(filename, df)
        df.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

### RS Rating and Ranking

In [26]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "Russell 1000", "PHLX Semiconductor", "All Indices"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'Russell 1000': 'RUI',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+RUI+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], period)
for df in (rank_stock, rank_indust):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

[*********************100%%**********************]  1003 of 1003 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFChartError('%ticker%: No data found, symbol may be delisted')


..................................................


/usr/local/lib/python3.10/dist-packages/vistock/ibd.py:217: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = closes.ffill().pct_change(periods=periods)




***
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "SPX+DJIA+NDX+RUI+SOX_stocks_2y_20240826.csv" is in the "out" folder.
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "SPX+DJIA+NDX+RUI+SOX_industries_2y_20240826.csv" is in the "out" folder.
***



,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
6,VKTX,64.81,Healthcare,Biotechnology,246.26,274.36,280.95,248.51,99,99,99,99,1
621,CVNA,155.09,Consumer Cyclical,Auto & Truck Dealerships,246.11,241.35,329.32,328.33,99,99,99,99,2
111,CAVA,126.10,Consumer Cyclical,Restaurants,213.97,142.26,151.32,105.63,99,98,98,72,3
43,MSTR,149.35,Technology,Software—Application,192.24,255.83,320.90,192.22,99,99,99,99,4
831,NVDA,126.21,Technology,Semiconductors,160.99,167.54,184.46,184.67,99,99,99,99,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,FIVN,33.41,Technology,Software—Infrastructure,48.46,60.92,66.24,77.21,0,2,3,9,998
182,WBA,10.34,Healthcare,Pharmaceutical Retailers,46.89,48.46,60.02,70.07,0,0,0,4,999
765,FIVE,83.59,Consumer Cyclical,Specialty Retail,45.31,36.38,61.27,88.03,0,0,1,32,1000
505,NFE,12.79,Utilities,Utilities—Regulated Gas,38.65,59.47,70.52,80.07,0,1,4,14,1001


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
109,Mortgage Finance,Financial Services,145.04,123.99,116.18,110.35,"RKT,UWMC",99,99,92,89,1
67,Utilities—Independent Power Producers,Utilities,141.95,134.26,214.47,131.38,"VST,NRG",99,99,99,98,2
19,Auto & Truck Dealerships,Consumer Cyclical,124.36,121.38,130.01,130.87,"CVNA,KMX,AN,LAD,VVV,PAG",98,97,97,97,3
112,Residential Construction,Consumer Cyclical,120.11,118.95,108.41,118.55,"TOL,DHI,PHM,NVR,LEN",97,96,84,94,4
82,Gold,Basic Materials,113.32,99.70,101.95,69.96,"NEM,RGLD",96,65,73,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
71,Staffing & Employment Services,Industrials,81.46,79.72,81.36,84.42,"MAN,RHI",3,0,5,16,116
69,Chemicals,Basic Materials,81.11,85.13,93.63,93.48,"DOW,CE,HUN",2,9,37,42,117
107,Grocery Stores,Consumer Defensive,79.93,81.47,83.49,83.41,"KR,ACI,GO",1,3,11,15,118
85,Resorts & Casinos,Consumer Cyclical,78.79,76.65,75.22,80.26,"BYD,PENN,CZR,MGM,MTN,LVS,WYNN,VAC",0,0,0,8,119


In [8]:
# @title Top Percentile Stocks
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
print(top_stock_list)


number of filtered tickers: 102
['CVNA', 'VKTX', 'CAVA', 'MSTR', 'NVDA', 'RKT', 'THC', 'VST', 'ALNY', 'COHR', 'SN', 'APP', 'COKE', 'NTRA', 'PLTR', 'IRM', 'HOOD', 'GDDY', 'VIRT', 'SPOT', 'NU', 'DKS', 'CRUS', 'UI', 'HWM', 'NRG', 'MMM', 'UHS', 'COIN', 'MHK', 'FICO', 'PPC', 'VNO', 'UWMC', 'WING', 'BURL', 'KKR', 'TPL', 'TRGP', 'WAL', 'JEF', 'MPWR', 'AXON', 'VNOM', 'TOL', 'GAP', 'NTAP', 'FRPT', 'EVR', 'JLL', 'UTHR', 'GE', 'DOCS', 'AVGO', 'RARE', 'HIW', 'DHI', 'GWRE', 'ANET', 'K', 'CFG', 'PGR', 'LAZ', 'GME', 'TSM', 'NEM', 'LLY', 'FYBR', 'CBRE', 'PHM', 'HRB', 'ALSN', 'PSTG', 'TPG', 'LPX', 'ISRG', 'BFAM', 'GRMN', 'EME', 'FIX', 'TRU', 'RCL', 'PSN', 'NVR', 'FITB', 'GM', 'HCA', 'RBLX', 'EFX', 'ALLY', 'SYF', 'VTR', 'CTAS', 'ATI', 'MSI', 'DVA', 'VRT', 'LII', 'TT', 'URI', 'META', 'MTZ']


In [9]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = filter_increasing_relative_strength(rank_stock)
data_table.DataTable(filtered_rank_stock, include_index=False, num_rows_per_page=10)

,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
927,RKT,20.95,Financial Services,Mortgage Finance,160.81,132.12,122.77,111.69,99,96,92,81,6
816,COKE,1353.90,Consumer Defensive,Beverages—Non-Alcoholic,141.99,131.85,115.48,107.68,98,96,87,75,13
237,VIRT,30.74,Financial Services,Capital Markets,136.58,123.62,113.00,79.90,98,93,84,13,19
193,UI,206.38,Technology,Communication Equipment,134.03,123.66,96.19,68.59,97,93,53,3,24
501,MMM,130.91,Industrials,Conglomerates,131.77,128.58,109.16,79.05,97,95,79,11,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,FE,43.47,Utilities,Utilities—Regulated Electric,100.81,94.02,91.85,82.45,61,44,41,18,393
343,DTE,124.71,Utilities,Utilities—Regulated Electric,100.49,95.42,92.56,84.54,60,48,43,23,398
579,SWK,100.96,Industrials,Tools & Accessories,100.34,92.68,86.65,85.23,60,39,26,25,401
641,WEC,92.93,Utilities,Utilities—Regulated Electric,100.29,86.82,86.30,76.43,60,23,25,8,404


### RS Rating and Ranking for Taiwan Stocks

In [30]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], period,
     tickers_getter=tw.get_tickers, ticker_ref='^TWII')
for df in (tw_stocks, tw_industries):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

[*********************100%%**********************]  2075 of 2075 completed
ERROR:yfinance:
69 Failed downloads:
ERROR:yfinance:['6957.TW', '00951.TW', '3150.TW', '00949.TW', '00947.TW', '6890.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']")
ERROR:yfinance:['02001L.TW', '020037.TW', '020029.TW', '020019.TW', '020020.TW', '020034.TW', '020030.TW', '02001R.TW', '020016.TW', '020038.TW', '020036.TW', '020039.TW', '020031.TW', '020028.TW', '020012.TW', '020011.TW', '020000.TW', '020015.TW', '02001S.TW', '020018.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d']")
ERROR:yfinance:['00956.TW', '00954.TW', '6838.TW', '00953B.TW']: YFInvalidPeriodError("%ticker%: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']")
ERROR:yfinance:['6949.TW', '6794.TW', '6928.TW', '00941.TW', '00940.TW', '00946.TW', '6423.TW', '4771.TW', '4949.TW', '00943.TW', '00939.TW', '6771.TW', 

..................................................


/usr/local/lib/python3.10/dist-packages/vistock/ibd.py:217: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret = closes.ffill().pct_change(periods=periods)




***
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "TWSE+TPEX_stocks_2y_20240826.csv" is in the "out" folder.
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "TWSE+TPEX_industries_2y_20240826.csv" is in the "out" folder.
***



,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
894,6144,105.50,Communication Services,Entertainment,549.43,454.28,138.54,91.45,97,97,92,43,1
996,1799,226.00,Healthcare,Drug Manufacturers—Specialty & Generic,431.52,325.99,281.87,193.94,97,97,97,95,2
43,6640,NaN,Technology,Semiconductors,424.27,416.15,249.07,236.44,97,97,97,96,3
1789,6442,442.00,Technology,Electronic Components,369.75,409.51,181.02,218.81,97,97,96,95,4
1615,8374,135.00,Industrials,Industrial Distribution,335.89,334.99,113.03,82.62,97,97,84,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,5227,30.00,Industrials,Electrical Equipment & Parts,56.48,65.63,56.30,69.75,0,1,0,1,2070
1372,1796,46.05,Healthcare,Drug Manufacturers—Specialty & Generic,56.11,71.07,64.37,58.46,0,6,2,0,2071
638,6958,30.95,Financial Services,Credit Services,55.87,74.24,73.18,87.36,0,11,12,30,2072
1194,2740,NaN,Consumer Cyclical,Restaurants,55.06,59.12,68.10,76.31,0,0,5,5,2073


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
75,Industrial Distribution,Industrials,147.25,149.27,93.17,88.81,"8374,911608,3114,2373",98,98,70,29,1
67,Utilities—Renewable,Utilities,139.56,137.70,98.84,113.93,"6869,6873,6806,8087",97,97,81,95,2
52,Entertainment,Communication Services,137.32,126.12,91.51,93.93,"6144,4806,6596,8446,8450,6184,8487,6625,6464,6856",96,93,64,58,3
56,Real Estate—Development,Real Estate,132.64,136.28,120.72,106.13,"2524,5508,3188,2537,1436,5455,2718,6171,3489,6...",95,96,98,82,4
92,Recreational Vehicles,Consumer Cyclical,125.70,101.28,78.06,96.11,"8937,9110,8478",94,84,13,65,5
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Apparel Retail,Consumer Cyclical,77.90,77.86,67.84,79.95,"1417,2911",4,6,1,3,91
53,Auto Manufacturers,Consumer Cyclical,75.21,76.68,79.42,93.94,"2206,2258,1599,2227,2201,2204",3,4,20,59,92
63,Travel Services,Consumer Cyclical,74.68,79.73,94.79,82.93,"2745,2731,5706,2743,2719,6242,2734",2,12,74,6,93
50,Pharmaceutical Retailers,Healthcare,73.47,71.55,70.88,85.46,"4173,4175,6469",1,0,3,18,94


In [11]:
# @title Top Percentile Taiwan Stocks
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}

top_stocks = tw_stocks[tw_stocks[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
print(top_stock_list)


number of filtered tickers: 63
['得利影', '易威', '均華', '光聖', '羅昇', '錦明', '慧友', '穎漢', '福裕', '皇昌', '京城', '均豪', '昆盈', '晶彩科', '擎亞', '康全電訊', '福大', '新復興', '海悅', '欣巴巴', '天方能源', '志聖', '所羅門', '新門', '彬台', '天揚', '弘憶股', '順藥', '高鋒', '翔耀', '太普高', '東捷', '旺矽', '弘塑', '福懋油', '合騏', '天品', '鑫科', '精湛', '訊舟', '昇陽半導體', '永信建', '雲豹能源', '世紀', '系微', '訊聯基因', '花王', '鏵友益', '慶騰', '鈊象', '力麗店', '晟銘電', '鑫龍騰', '安國', '聯鈞', '泰金-KY', '德晉', '藝舍-KY', '峰源-KY', '泓德能源-創', '錸德', '精材', '喬福']


In [12]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

def convert_to_list(value):
    if isinstance(value, str):
        return value.split(',')
    return value

filtered_tw_stocks = filter_increasing_relative_strength(tw_stocks)
update_tickers_with_names(filtered_tw_stocks, 'Ticker')

filtered_tw_industries = filter_increasing_relative_strength(tw_industries)
update_tickers_with_names(filtered_tw_industries, 'Tickers')
#filtered_tw_industries.loc[:, 'Tickers'] = filtered_tw_industries['Tickers'].apply(convert_to_list)

for df in (filtered_tw_stocks, filtered_tw_industries):
    display(data_table.DataTable(df, include_index=False, num_rows_per_page=10))

,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
894,得利影,105.50,Communication Services,Entertainment,547.41,454.28,138.54,91.45,97,97,92,43,1
996,易威,226.00,Healthcare,Drug Manufacturers—Specialty & Generic,429.93,325.99,281.87,193.94,97,97,97,95,2
43,均華,968.00,Technology,Semiconductors,422.70,416.15,249.07,236.44,97,97,97,96,3
1806,錦明,42.00,Technology,Electronic Components,297.92,173.51,96.58,93.29,97,95,69,48,6
1321,福裕,75.10,Industrials,Specialty Industrial Machinery,286.93,178.23,153.96,86.83,97,95,94,28,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,國泰台灣領袖50,21.42,NaN,NaN,101.23,101.04,99.42,95.70,72,73,72,54,529
1301,勝品,114.00,Industrials,Security & Protection Services,101.17,93.62,84.40,74.10,72,62,44,3,530
1074,永豐臺灣加權,112.55,NaN,NaN,100.40,98.89,98.78,98.29,71,70,72,58,549
1664,元大MSCI金融,29.00,NaN,NaN,100.37,96.56,89.82,89.47,71,67,58,38,550


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
72,Insurance—Life,Financial Services,102.17,93.72,86.45,84.16,"新光金,三商壽,三商,富邦金,新光金乙特,新光金甲特,國泰金,開發金,開發金乙特,國泰特,富...",79,62,46,8,20


### Remove files in GitHub Repository

In [13]:
# @title CSV Deleter
import re
import ipywidgets as widgets

# Example filenames
with requests_cache.disabled():
    all_filenames = github.list_filenames()

# Function to extract unique dates from filenames
def extract_dates(filenames):
    date_pattern = r'\d{8}'
    dates = set()
    for fn in filenames:
        match = re.search(date_pattern, fn)
        if match:
            dates.add(match.group(0))
    return sorted(dates, reverse=True)  # Sort dates from newest to oldest

# Function to remove a file (replace with your actual implementation)
def remove_file(filename):
    print(f"Removing file: {filename}")
    with requests_cache.disabled():
        github.remove_file(filename)
    all_filenames.remove(filename)

#-------------------------------------------------------------------------------

# Update file selector options based on selected date
def update_file_selector(change):
    def selector_width(filenames):
        max_filename_length = max(len(fn) for fn in filenames)
        return f'{max_filename_length * 10}px'  # 10px width per character

    selected_date = change['new']
    lst_fns = [fn for fn in all_filenames if selected_date in fn]
    file_selector_widget.options = lst_fns
    file_selector_widget.rows = len(file_selector_widget.options)
    file_selector_widget.layout=widgets.Layout(width=selector_width(lst_fns))

# Function to delete selected files
def delete_files(button):
    selected_files = file_selector_widget.value
    for file in selected_files:
        remove_file(file)
    update_widgets()    # Update widgets after deletion

# Update widgets to reflect current state
def update_widgets():
    # Refresh the date selector
    dates = extract_dates(all_filenames)
    selected_date = date_selector_widget.value
    if dates and selected_date not in dates:
        i = date_selector_widget.options.index(selected_date)
        if i > len(dates) - 1:
            selected_date = dates[-1]
        else:
            selected_date = dates[i]
    date_selector_widget.options = dates
    date_selector_widget.value = selected_date
    if not dates:
        file_selector_widget.options = []
        return
    update_file_selector({'new': date_selector_widget.value})

#-------------------------------------------------------------------------------

# Create a widget for selecting dates
def create_date_selector(dates):
    return widgets.Dropdown(
        options=dates,
        value = dates[0] if dates else None,
        description='Date:',
        disabled=False
    )

# Create a widget for selecting files
def create_file_selector(filenames):

    return widgets.SelectMultiple(
        options=[],
        value=[],
        description='Files',
        disabled=False,
    )

# Create widgets
dates = extract_dates(all_filenames)
date_selector_widget = create_date_selector(dates)
file_selector_widget = create_file_selector(all_filenames)
delete_button = widgets.Button(description="Delete Selected Files")
delete_button.on_click(delete_files)

# Initialize the file selector with the latest date
update_widgets()

# Set up the observer to update file selector when date is changed
date_selector_widget.observe(update_file_selector, names='value')

# Display widgets
display(date_selector_widget)
display(file_selector_widget)
display(delete_button)


Dropdown(description='Date:', options=('20240826',), value='20240826')

SelectMultiple(description='Files', layout=Layout(width='470px'), options=('SPX+DJIA+NDX+RUI+SOX_industries_2y…

Button(description='Delete Selected Files', style=ButtonStyle())